In [1]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import LayerNormalization
import tensorflow as tf
from keras import backend as K
from sklearn.model_selection import train_test_split 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
X = np.load('/content/drive/MyDrive/Capstone Project EDA/X_data.npy', allow_pickle = True)
y = np.load('/content/drive/MyDrive/Capstone Project EDA/Y_data.npy', allow_pickle = True)

In [5]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping: no known devices.


In [6]:
r = np.random.permutation(len(X))
X = X[r]
y = y[r]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
weight_for_0 = (1 / (len(y_train)-sum(y_train))) * (len(y_train) / 2.0)
weight_for_1 = (1 / sum(y_train)) * (len(y_train) / 2.0)
class_weight = {0:weight_for_0,1:weight_for_1}

In [ ]:
#Model

model = Sequential()

model.add(LayerNormalization(axis=-1 , center=True , scale=True))
model.add(Bidirectional(LSTM(100, return_sequences=True,stateful=False)))
model.add(LayerNormalization(axis=-1 , center=True , scale=True))
model.add(Bidirectional(LSTM(50, return_sequences=True,stateful=False)))
model.add(LayerNormalization(axis=-1, center=True , scale=True))
model.add(Bidirectional(LSTM(25, return_sequences=False,stateful=False)))
model.add(LayerNormalization(axis=-1, center=True , scale=True))

model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','AUC'])
# train LSTM
model.fit(X_train, y_train, epochs=75, batch_size=32, verbose=2, class_weight=class_weight)
# evaluate LSTM
yhat = model.predict(X_test, verbose=2)
model.evaluate(X_test,y_test)


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/75


In [ ]:
y_train

array([0., 0., 0., ..., 1., 0., 0.])

In [ ]:
yhat.shape

(38615,)

In [ ]:
np.argmax(yhat,axis=1)

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
sum(yhat)

array([11403.37], dtype=float32)